In [1]:
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns 
import numpy as np
from copy import deepcopy as cdc
import prey_capture_python as preycap
import pathlib as pl
import glob
import os
from operator import itemgetter
from itertools import groupby

In [2]:
datacsv='/Volumes/Projects/PreyCapture/ZIActivation/DataDirs_newcohort.csv'

In [73]:
files_df=pd.read_csv(datacsv)

In [75]:
datacsv='/Volumes/Projects/PreyCapture/ZIActivation/DataDirs_newcohort.csv'
files_df=pd.read_csv(datacsv)
list_of_nofilter=[]
new_stem = "/Volumes/Projects/PreyCapture/ZIActivation/"

for i, row in files_df.iterrows():
    file = row["path"]
    path_parts = list(pl.PureWindowsPath(file).parts)
    path_parts = [path_parts[-1]]
    for path in path_parts:
        path=str(new_stem+path)
        os.chdir(path)
        try:
            csv=glob.glob('*_filtered.csv')
            csv[0]
            dates=[]
            for j, file in enumerate(csv):
                dates.append(pl.Path(str(file)).stat().st_mtime)
            index=np.argmax(dates)
            csv_new=csv[index]
            posix_csv_path=str(path+'/'+csv_new)
        except IndexError:
            posix_csv_path=np.nan
            list_of_nofilter.append(path)
            print("{} likely has no filtered csv. please manually check".format(path))

    path_parts_og = cdc(path_parts)

    files_df.at[i, "posix_csv_path"] = posix_csv_path
    folder_path = pl.Path(*path_parts_og)

/Volumes/Projects/PreyCapture/ZIActivation/2022-12-07_11-55-35_mouse-1430 likely has no filtered csv. please manually check
/Volumes/Projects/PreyCapture/ZIActivation/2022-12-09_14-01-25_mouse-1335 likely has no filtered csv. please manually check
/Volumes/Projects/PreyCapture/ZIActivation/2023-01-11_12-44-38_mouse-1430 likely has no filtered csv. please manually check


In [6]:
files_df

,path,Cond,FatalFlag,Laser,posix_csv_path
0,/Volumes/Projects/PreyCapture/ZIActivation/202...,test,NaN,0.0,/Volumes/Projects/PreyCapture/ZIActivation/202...
1,/Volumes/Projects/PreyCapture/ZIActivation/202...,test,NaN,0.0,/Volumes/Projects/PreyCapture/ZIActivation/202...
2,/Volumes/Projects/PreyCapture/ZIActivation/202...,test,NaN,0.0,/Volumes/Projects/PreyCapture/ZIActivation/202...
3,/Volumes/Projects/PreyCapture/ZIActivation/202...,test,NaN,0.0,/Volumes/Projects/PreyCapture/ZIActivation/202...
4,/Volumes/Projects/PreyCapture/ZIActivation/202...,test,NaN,0.0,/Volumes/Projects/PreyCapture/ZIActivation/202...
...,...,...,...,...,...
1295,/Volumes/Projects/PreyCapture/ZIActivation/202...,9,NaN,1.0,/Volumes/Projects/PreyCapture/ZIActivation/202...
1296,/Volumes/Projects/PreyCapture/ZIActivation/202...,9,NaN,0.0,/Volumes/Projects/PreyCapture/ZIActivation/202...
1297,/Volumes/Projects/PreyCapture/ZIActivation/202...,9,NaN,1.0,/Volumes/Projects/PreyCapture/ZIActivation/202...
1298,/Volumes/Projects/PreyCapture/ZIActivation/202...,9,NaN,0.0,/Volumes/Projects/PreyCapture/ZIActivation/202...


In [7]:
file_list=files_df['posix_csv_path'].to_numpy()


In [106]:
file_list[0]

'/Volumes/Projects/PreyCapture/ZIActivation/2022-12-07_11-20-50_mouse-1430/Sky_mouse-1430_2022-12-07T11_20_50DLC_resnet50_optopreycapFeb16shuffle1_300000_el_filtered.csv'

In [8]:
def check_tracks(file_list, saveflag, plotter):
    mouse_xy, cricket_p, cricket_xy, rear_xy, lear_xy, headbase_xy, cricket_front, cricket_back=preycap.extract_points(file, ['Rear', 'Lear', 'anteriorC', 'posteriorC','headbase'])
    mouse_gaps=np.where(np.isnan(mouse_xy))
    cricket_gaps=np.where(np.isnan(cricket_xy))
    problem_mouse=[]
    path = pl.PurePath(file)
    if len(mouse_gaps[0])>400:


        print("{} is missing mouse tracks, check manually".format(path.parent.name))
        problem_mouse=file
        if plotter==1 and saveflag==1:
            plt.figure(figsize=(10,10))
            plt.plot(mouse_xy[0], mouse_xy[1], linestyle="", marker='.', markersize=1)
            plt.plot(cricket_xy[0], cricket_xy[1], linestyle="", marker='.', markersize=1)
            plt.title('Mouse and Cricket Tracks, {}'.format(path.parent.name))
            sns.despine()
            plt.savefig('/Users/mollyshallow/Desktop/check_dlctracks/plots/tracks_{}.pdf'.format(path.parent.name))
            np.save('/Users/mollyshallow/Desktop/check_dlctracks/mouse_gaps/mouse_gaps_{}.npy'.format(path.parent.name), mouse_gaps)
            np.save('/Users/mollyshallow/Desktop/check_dlctracks/cricket_gaps/cricket_gaps_{}.npy'.format(path.parent.name), cricket_gaps)
    else:
        problem_mouse=np.nan


    return mouse_gaps, cricket_gaps, problem_mouse

In [153]:
def consecutive_drops(problem_files):
    trials_to_drop=[]
    for j in problem_files:
        try:
            path = pl.PurePath(j)
            # print(path)
            gaps=np.load('/Users/mollyshallow/Desktop/check_dlctracks/mouse_gaps/mouse_gaps_'+path.parent.name+'.npy')
            # print(gaps)
            gap_ranges =[]
            for k,g in groupby(enumerate(gaps[1]),lambda x:x[0]-x[1]):
                group = (map(itemgetter(1),g))
                group = list(map(int,group))
                gap_ranges.append((group[0],group[-1]))
            len_gaps=np.diff(gap_ranges)
            if any(len_gaps>600):
                print('{} is missing the mouse for more than 3 sec'.format(path.parent.name))
                trials_to_drop.append(path)

        except (TypeError, FileNotFoundError):
            print('no mouse gaps file')
    return trials_to_drop

In [107]:
test_list=file_list[0:10]

In [154]:
%matplotlib agg
%matplotlib agg
# list_of_problem_mice=[]

# for file in file_list:
#     if file is not np.nan:
#         mouse_gaps, cricket_gaps, problem_mouse= check_tracks(file, saveflag=0, plotter=0)
#         list_of_problem_mice.append(problem_mouse)
# list_of_problem_mice=np.asarray(list_of_problem_mice)
# mice_to_save = pd.DataFrame(list_of_problem_mice, columns=['filename'])
# mice_to_save.to_csv("/Users/mollyshallow/Desktop/check_dlctracks/problem_mice_newcohort4.csv")

#pull .npy files for remaining problem trials, figure out if drops are consecutive, make list of files to be dropped
problem_files=pd.read_csv('/Users/mollyshallow/Desktop/check_dlctracks/problem_mice_newcohort4.csv')
problem_files=problem_files['filename'].to_numpy()

trials_to_drop=consecutive_drops(problem_files)

trials_to_drop=np.asarray(trials_to_drop)
trials_to_drop=pd.DataFrame(trials_to_drop, columns=['posix_csv_path'])
trials_to_drop=trials_to_drop['posix_csv_path'].astype(str)
trials_to_drop.to_csv("/Users/mollyshallow/Desktop/check_dlctracks/trials_to_drop_newcohort3.csv")

#maybe validate with plot/ video

#match up these trials to the dataframe of conditions, check to make sure they aren't exploration trials -- not sure if this is necessary 
files_df_wdrops=pd.merge(files_df,trials_to_drop, how='outer', on='posix_csv_path', indicator=True)
files_df_wdrops=files_df_wdrops.rename(columns={"_merge": "trials_to_drop"})
files_df_wdrops.to_csv('/Volumes/Projects/PreyCapture/ZIActivation/DataDirs_newcohort_wdrops.csv')
#check and remove exploration trials, then figure out what to do with the ones actually missing cricket points


no mouse gaps file
no mouse gaps file
no mouse gaps file
2022-12-07_11-43-38_mouse-1430 is missing the mouse for more than 3 sec
2022-12-07_11-49-41_mouse-1430 is missing the mouse for more than 3 sec
2022-12-07_12-00-33_mouse-1430 is missing the mouse for more than 3 sec
no mouse gaps file
2022-12-07_12-57-23_mouse-1430 is missing the mouse for more than 3 sec
no mouse gaps file
no mouse gaps file
no mouse gaps file
2022-12-07_13-16-51_mouse-1429 is missing the mouse for more than 3 sec
2022-12-07_13-21-18_mouse-1429 is missing the mouse for more than 3 sec
no mouse gaps file
no mouse gaps file
no mouse gaps file
2022-12-07_13-30-18_mouse-1429 is missing the mouse for more than 3 sec
2022-12-07_13-32-35_mouse-1429 is missing the mouse for more than 3 sec
2022-12-07_18-06-15_mouse-1336 is missing the mouse for more than 3 sec
no mouse gaps file
no mouse gaps file
2022-12-07_18-20-57_mouse-1336 is missing the mouse for more than 3 sec
no mouse gaps file
2022-12-07_18-33-11_mouse-1336 is

In [106]:
trials_to_drop
trials_to_drop=trials_to_drop.rename(columns={"filename": "posix_csv_path"})

In [89]:
files_df

,path,Cond,FatalFlag,Laser,posix_csv_path
0,/Volumes/Projects/PreyCapture/ZIActivation/202...,test,NaN,0.0,/Volumes/Projects/PreyCapture/ZIActivation/202...
1,/Volumes/Projects/PreyCapture/ZIActivation/202...,test,NaN,0.0,/Volumes/Projects/PreyCapture/ZIActivation/202...
2,/Volumes/Projects/PreyCapture/ZIActivation/202...,test,NaN,0.0,/Volumes/Projects/PreyCapture/ZIActivation/202...
3,/Volumes/Projects/PreyCapture/ZIActivation/202...,test,NaN,0.0,/Volumes/Projects/PreyCapture/ZIActivation/202...
4,/Volumes/Projects/PreyCapture/ZIActivation/202...,test,NaN,0.0,/Volumes/Projects/PreyCapture/ZIActivation/202...
...,...,...,...,...,...
1295,/Volumes/Projects/PreyCapture/ZIActivation/202...,9,NaN,1.0,/Volumes/Projects/PreyCapture/ZIActivation/202...
1296,/Volumes/Projects/PreyCapture/ZIActivation/202...,9,NaN,0.0,/Volumes/Projects/PreyCapture/ZIActivation/202...
1297,/Volumes/Projects/PreyCapture/ZIActivation/202...,9,NaN,1.0,/Volumes/Projects/PreyCapture/ZIActivation/202...
1298,/Volumes/Projects/PreyCapture/ZIActivation/202...,9,NaN,0.0,/Volumes/Projects/PreyCapture/ZIActivation/202...


In [139]:
trials_to_drop=trials_to_drop['posix_csv_path'].astype(str)
type(trials_to_drop[0])

str

In [140]:
test=pd.merge(files_df,trials_to_drop, how='outer', on='posix_csv_path', indicator=True)

In [149]:
files_df_wdrops=test.rename(columns={"_merge": "trials_to_drop"})
files_df_wdrops

,path,Cond,FatalFlag,Laser,posix_csv_path,trials_to_drop
0,/Volumes/Projects/PreyCapture/ZIActivation/202...,test,NaN,0.0,/Volumes/Projects/PreyCapture/ZIActivation/202...,left_only
1,/Volumes/Projects/PreyCapture/ZIActivation/202...,test,NaN,0.0,/Volumes/Projects/PreyCapture/ZIActivation/202...,left_only
2,/Volumes/Projects/PreyCapture/ZIActivation/202...,test,NaN,0.0,/Volumes/Projects/PreyCapture/ZIActivation/202...,left_only
3,/Volumes/Projects/PreyCapture/ZIActivation/202...,test,NaN,0.0,/Volumes/Projects/PreyCapture/ZIActivation/202...,left_only
4,/Volumes/Projects/PreyCapture/ZIActivation/202...,test,NaN,0.0,/Volumes/Projects/PreyCapture/ZIActivation/202...,left_only
...,...,...,...,...,...,...
1295,/Volumes/Projects/PreyCapture/ZIActivation/202...,9,NaN,1.0,/Volumes/Projects/PreyCapture/ZIActivation/202...,left_only
1296,/Volumes/Projects/PreyCapture/ZIActivation/202...,9,NaN,0.0,/Volumes/Projects/PreyCapture/ZIActivation/202...,left_only
1297,/Volumes/Projects/PreyCapture/ZIActivation/202...,9,NaN,1.0,/Volumes/Projects/PreyCapture/ZIActivation/202...,left_only
1298,/Volumes/Projects/PreyCapture/ZIActivation/202...,9,NaN,0.0,/Volumes/Projects/PreyCapture/ZIActivation/202...,left_only


In [145]:
len(np.where(test['_merge'].to_numpy()=='both')[0])

197

In [35]:
str(trials_to_drop['filename'][0])

'/Volumes/Projects/PreyCapture/ZIActivation/2022-12-07_11-43-38_mouse-1430/Sky_mouse-1430_2022-12-07T11_43_38DLC_resnet50_optopreycapFeb16shuffle1_300000_el_filtered.csv'

In [45]:
files_df['posix_csv_path'].str.contains(str(trials_to_drop['filename'][0])).any()

True

KeyError: "None of [Index(['/Volumes/Projects/PreyCapture/ZIActivation/2022-12-07_11-20-50_mouse-1430',\n       '/Volumes/Projects/PreyCapture/ZIActivation/2022-12-07_11-22-51_mouse-1430',\n       '/Volumes/Projects/PreyCapture/ZIActivation/2022-12-07_11-28-25_mouse-1430',\n       '/Volumes/Projects/PreyCapture/ZIActivation/2022-12-07_11-33-52_mouse-1430',\n       '/Volumes/Projects/PreyCapture/ZIActivation/2022-12-07_11-38-49_mouse-1430',\n       '/Volumes/Projects/PreyCapture/ZIActivation/2022-12-07_11-43-38_mouse-1430',\n       '/Volumes/Projects/PreyCapture/ZIActivation/2022-12-07_11-49-41_mouse-1430',\n       '/Volumes/Projects/PreyCapture/ZIActivation/2022-12-07_11-55-35_mouse-1430',\n       '/Volumes/Projects/PreyCapture/ZIActivation/2022-12-07_12-00-33_mouse-1430',\n       '/Volumes/Projects/PreyCapture/ZIActivation/2022-12-07_12-53-48_mouse-1430',\n       ...\n       '/Volumes/Projects/PreyCapture/ZIActivation/2023-01-29_13-13-59_mouse-1430',\n       '/Volumes/Projects/PreyCap

In [69]:
missing

array([0])

In [72]:
test=files_df['missing_tracks'].to_numpy()
np.where(test==0)

(array([], dtype=int64),)

In [76]:
files_df

,path,Cond,FatalFlag,Laser,posix_csv_path
0,/Volumes/Projects/PreyCapture/ZIActivation/202...,test,NaN,0.0,/Volumes/Projects/PreyCapture/ZIActivation/202...
1,/Volumes/Projects/PreyCapture/ZIActivation/202...,test,NaN,0.0,/Volumes/Projects/PreyCapture/ZIActivation/202...
2,/Volumes/Projects/PreyCapture/ZIActivation/202...,test,NaN,0.0,/Volumes/Projects/PreyCapture/ZIActivation/202...
3,/Volumes/Projects/PreyCapture/ZIActivation/202...,test,NaN,0.0,/Volumes/Projects/PreyCapture/ZIActivation/202...
4,/Volumes/Projects/PreyCapture/ZIActivation/202...,test,NaN,0.0,/Volumes/Projects/PreyCapture/ZIActivation/202...
...,...,...,...,...,...
1295,/Volumes/Projects/PreyCapture/ZIActivation/202...,9,NaN,1.0,/Volumes/Projects/PreyCapture/ZIActivation/202...
1296,/Volumes/Projects/PreyCapture/ZIActivation/202...,9,NaN,0.0,/Volumes/Projects/PreyCapture/ZIActivation/202...
1297,/Volumes/Projects/PreyCapture/ZIActivation/202...,9,NaN,1.0,/Volumes/Projects/PreyCapture/ZIActivation/202...
1298,/Volumes/Projects/PreyCapture/ZIActivation/202...,9,NaN,0.0,/Volumes/Projects/PreyCapture/ZIActivation/202...


In [30]:
no_cricket_trials=pd.read_csv("/Users/mollyshallow/Desktop/20230501_failed_newmice.csv", usecols=[1,2])
problem_files=pd.read_csv('/Users/mollyshallow/Desktop/check_dlctracks/problem_mice_newcohort2.csv')
problem_files=problem_files['filename'].to_numpy()

missing_cricket=[]
for i in problem_files:
    if i is not np.nan and no_cricket_trials['1'].str.contains(i).any() == True:
        path = pl.PurePath(i)
        missing_cricket.append(i)
        print("{} is missing mouse and cricket tracks, check manually".format(path.parent.name))
        

2022-12-07_12-57-23_mouse-1430 is missing mouse and cricket tracks, check manually


In [34]:
file='/Volumes/Projects/PreyCapture/ZIActivation/2022-12-07_11-43-38_mouse-1430/Sky_mouse-1430_2022-12-07T11_43_38DLC_dlcrnetms5_optopreycapFeb16shuffle1_150000_el_filtered.csv'
mouse_gaps, cricket_gaps, problem_mouse= check_tracks(file, saveflag=0, plotter=0)


2022-12-07_11-43-38_mouse-1430 is missing mouse tracks, check manually
